##Explore: himalaya.bronze.deaths
Goal: Understand the structure, quality, and issues in the deaths dataset before Silver transformation.

###_Previewing Data_

In [0]:
import pyspark.sql.functions as F

In [0]:
df = spark.table("himalaya.bronze.deaths")

In [0]:
print(f"Rows: {df.count()}")
df.printSchema()
df.show(10)

###_Finding Errors_
1. Null
2. Duplicates
3. Distinct
4. Format
5. Timeliness

> ## 1. Nulls

In [0]:
for c in df.columns:
  null_count = df.filter(F.col(c).isNull()).count()
  status = "✅" if null_count == 0 else "❌"
  print(f"{status} Nulls in {c}: {null_count}")

In [0]:
for c in df.columns:
  null_count = df.filter(F.col(c).isNull()).count()
  if null_count == 0:
      print(f"{c}:")
      df.filter(F.col("nationality").isNull()).show()

####_Nationality_
Can probably find out null values from other data sets.
If not in other data sets, will remove
___
####_Cause of Death_
No need to find out, it is ok if we don't know how they died

> ## 2. Duplicates

In [0]:
total = df.count()
distinct = df.distinct().count()
duplicates = total - distinct

print(f"Total rows:    {total}")
print(f"Distinct rows: {distinct}")
print(f"Duplicates:    {duplicates}")

if duplicates == 0:
    print("✅ No duplicates")
else:
    print(f"❌ {duplicates} duplicate found")

> ## 3. Distinct

In [0]:
cols = ["nationality", "cause_of_death", "mountain"]

for c in distinct:
    distinct_count = df.select(c).distinct().count()
    print(f"{c}: {distinct_count} distinct values")

In [0]:
nations = df.select(cols[0]).distinct().show(1000, truncate=False)

####⚠️ Nations 
some weird countries

In [0]:
cause = df.select(cols[1]).distinct().show(1000, truncate=False)

####⚠️ Cause of Deaths 
Various redundant.
Make categories.

In [0]:
mountain = df.select(cols[2]).distinct().show(1000, truncate=False)

####✅ Mountains

> ## 4. Format

In [0]:
df.printSchema()

###⚠️ Date
Change date datatype from **string** to **date**

>## 5. Timeliness

In [0]:
min_date, max_date = df.select(F.min("date"), F.max("date")).first()

today = date.today()

print(f"Min: {min_date}")
print(f"Max: {max_date}")
print(f"Today: {today}")

print("✅ Max date is not in the future" if str(max_date) <= str(today) else "❌ Max date is in the future")